### Initialization

In [2]:
# Basics
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# Netscraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # Google chrome is required to do the netscraping
from bs4 import BeautifulSoup
import time
import re

In [3]:
# Unfolds the whole dataframe, instead of a section
pd.set_option('display.max_rows', None) # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_colwidth', None) # If your DataFrame has long text values, widen the column display

### Data load

#### Update attention
Reach out to DTU IT Service and ask for a list of what courses the D&I Master students have completed during the last semester.
The format of the new data must be in accordance to the following guidelines:
- It must be named "data_new.xlsx"
- It must contain three columns with values in the order: "Term" (A), "Course code" (B), "Passed DI" (C) (See "Data/data_original.xlsx")
- Any anonymized data in the column "Passed DI" should be set to "<=5", assuming it is <=5. Other cases will need a workaround in the "Data filtering"-section.

In [6]:
# Loads the the existing dataset
df = pd.read_excel("Data/data.xlsx")

In [7]:
# Loads the new data
if os.path.isfile("Data/data_new.xlsx"):
    df_new = pd.read_excel("Data/data_new.xlsx") 
    print("data_new.xlsx is loaded.")
else:
    print("ERROR! \ndata_new.xlsx does not exist.")

ERROR! 
data_new.xlsx does not exist.


In [8]:
if "df_new" in globals():
    df_new.columns = ["Term", "Course code", "Passed DI"] # Sets column indexes for data_new
    df = pd.concat([df, df_new], ignore_index=True) # Appends the new data to df
    df_new # Shows df_new

In [9]:
# Saves updated data.xlsx and deletes data_new.xlsx, resetting the repository format
df.to_excel('Data/data.xlsx', index=False) # Overwrites the original "data.xlsx" with the updated data

if os.path.isfile("Data/data_new.xlsx"): # Deletes data_new
    os.remove("Data/data_new.xlsx")
    print("data.xlsx has been updated\ndata_new.xlsx has been deleted")

### Data filtering

In [11]:
# Removes special courses
df = df[~df["Course code"].astype(str).str.startswith("E")] # Removes fall special courses
df = df[~df["Course code"].astype(str).str.startswith("S")] # Removes spring special courses

In [12]:
# Corrects courses with xxxxx-x format to xxxxx format
df["Course code"] = df["Course code"].astype(str).str.replace(r"^(\d{5})-\d+$", r"\1", regex=True)

In [13]:
# Removes duplicates of courses, leaving only the newest of each course code
def term_to_sort_value(term):
    if isinstance(term, str) and re.match(r'^[EF]\d{2}$', term):
        year = int(term[1:])
        season = 0.5 if term[0] == 'E' else 0.0  # E (Autumn) is newer
        return year + season
    return -1  # Treat invalid/missing terms as oldest

# Create a helper column for sorting
df['term_sort'] = df['Term'].apply(term_to_sort_value)

# Sort so newest terms come first
df = df.sort_values(by='term_sort', ascending=False)

# Drop duplicates, keeping the most recent per 'Course code'
df = df.drop_duplicates(subset='Course code', keep='first')

# Removes the helper column
df = df.drop(columns='term_sort')

#### Update attention
The next cell requries evaluations each semester from the [MSc. D&I curriculum](https://www.dtu.dk/english/education/graduate/msc-programmes/design-and-innovation/curriculum) and [BSc. D&I curriculum](https://student.dtu.dk/studieordninger/bachelor/design-og-innovation/studieplan), where it should be assesed which courses have been added or removed from the curriculum.

In [15]:
# Defines the curriculum for DI MSc. students and the correlated bachelor's degree
mandatory = ["12100", "12106", "12105", "12101", "42505", "42503", "42502", # Polytechnical foundation
             "41636", "41639", "02809", "38110"] # Programme specific

semi = ["02830", "34365", "34366", "34367", "38113", "41073", "41083", "41084", "41635", "41637", "41638", "63851", # Course Group 1: Systematic Design
       "02266", "02806", "02808", "12240", "12250", "12772", "38102", "38103", "38108"] # Course Group 2: Design for People & Society

bsc = ["42500", "01001", "01002", "02002", "02003", "02402", "10060", "26020", "26022", "26021", "27020", "42620", "42622", # Polytechnical foundation
      "41011", "41012", "41013", "41028", "41029", "41031", "41046", "41051", "41612", "41615", "41020"] # Programme specific

In [16]:
# Removes curriculum defined courses, leaving only electives
df = df[~df['Course code'].isin(mandatory)] # Removes mandatory courses
df = df[~df['Course code'].isin(semi)] # Removes semi-mandatory courses
df = df[~df['Course code'].isin(bsc)] # Removes bachelor courses

In [17]:
# Defines the anonymized data in df[Passed DI] = 3, in the middle of the range [1:5]
df["Passed DI"] = df["Passed DI"].astype(str)
df.loc[df["Passed DI"] == "<=5", "Passed DI"] = 3

In [18]:
# Resets the row index
df = df.reset_index(drop=True)

### Netscraping

#### Update attention
If the URL format of the DTU course base changes, the urls in the netscraping section must be changed in accordance. The URL formats vary quite a bit, so it is recommenable to check the grade_URL and eval_URL functions if any issues arise.

In [21]:
# Adds empty columns for the netscraping values
df[["Course name",
    "Registered students",
    "Passed students",
    "Exam pass percentage",
    "Registered students",
    "Average grade",
    "ECTS",
    "Schedule",
    "Level",
    "URL",
    "Grade_URL",
    "Eval_URL",
    "Eval1",
    "Eval2",
    "Eval3",
    "Eval4",
    "Eval5",
    "Workload"]] = ""

In [22]:
# Adds values to the URL column
for i in range(len(df)):
    url = f"https://kurser.dtu.dk/course/{df.iloc[i]["Course code"]}" # Defines a URL
    df.loc[i,"URL"] = url # Assigns the URL

In [23]:
# Defines the extract_html function that can load each course site
def extract_html(url):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    
    # Extracts html from the url and makes it readable
    driver.get(url)
    time.sleep(2)  # Let JS load
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    
    return soup

In [24]:
# Removes outdated courses, then adds values (course name, ECTS, schedule, and level) to the existing ones

for i in tqdm(range(len(df))):
    url = df.loc[i,"URL"]
    soup = extract_html(url)
    title_tag = soup.title
    
    if title_tag is None or not title_tag.string or title_tag.string.strip() == "": #Eliminates outdated courses
        df = df.drop(i)
        
    else:
        # Adds course name
        titles = soup.find_all("title")
        for title in titles:
            name = title.get_text()
        df.loc[i, "Course name"] = name
        
        # Adds ECTS points
        label_cell = soup.find('td', string=lambda text: text and "Point( ECTS )" in text)
        ects = label_cell.find_next_sibling('td').text.strip()
        df.loc[i, "ECTS"] = ects

        # Adds Schedule
        label_cell = soup.find('td', string=lambda text: text and "Schedule" in text)
        schedule = label_cell.find_next_sibling('td').text.strip()
        df.loc[i, "Schedule"] = schedule
        
        # Adds course level (BSc, MSc)
        label_cell = soup.find('label', string=lambda text: text and "Course type" in text)
        course_type_td = label_cell.find_parent('td').find_next_sibling('td')
        course_type = course_type_td.find_all('div')[0].text.strip()
        df.loc[i, "Level"] = course_type

df = df.reset_index(drop=True) #Resets the index after outdated courses were removed

  0%|          | 0/184 [00:00<?, ?it/s]

In [25]:
# Schedule formatting

mapping = {
    "August": "August 3-week",
    "June": "June 3-week",
    "July": "July 3-week",
    "January": "January 3-week"
}

for i in tqdm(range(len(df))):
    value = str(df.loc[i, "Schedule"])
    output = []

    has_e_number = bool(re.search(r'\bE[0-9][AB]?\b', value))
    has_f_number = bool(re.search(r'\bF[0-9][AB]?\b', value))

    # Case 1: E followed by digit
    if has_e_number:
        output.append("Autumn 13-week")
    # Case 2: "Autumn" present and no E+digit
    elif "Autumn" in value:
        output.append("Autumn 13-week")

    # Case 3: F followed by digit
    if has_f_number:
        output.append("Spring 13-week")
    # Case 4: "Spring" present and no F+digit
    elif "Spring" in value:
        output.append("Spring 13-week")

    # 3-week courses
    for season in mapping:
        if season in value:
            output.append(mapping[season])

    # Converts multi-schedule courses form multiple strings to a single comma-seperated string
    df.loc[i, "Schedule"] = ", ".join(output) if output else np.nan 

  0%|          | 0/109 [00:00<?, ?it/s]

In [26]:
# Splits and explodes the df whenever a course has multiple schedule placements, creating multiple versions of the same course
df['Schedule'] = df['Schedule'].fillna('').apply(lambda x: [s.strip() for s in x.split(',') if s.strip()])
df = df.explode('Schedule').reset_index(drop=True)

In [27]:
# Eval_URL function
def eval_URL(soup):
    indicator1 = soup.find("div", class_="bar", string=lambda s: s and "Evaluations" in s)
    if indicator1:
        sibling1 = indicator1.find_next_sibling("div")
        
        if sibling1:
            eval_term1 = sibling1.get_text(strip=True)[-8]
            eval_term2 = sibling1.get_text(strip=True)[-11]
            course_term = df.loc[i,"Schedule"][0:3]
            
            if ((course_term in ["Aut", "Jan"] and (eval_term1 == "E" or eval_term2 == "E")) or
                (course_term in ["Spr", "Aug", "Jun"] and (eval_term1 == "F" or eval_term2 == "F"))):
                link1 = sibling1.find("a")
                if link1 and link1.has_attr("href"):
                    df.loc[i, "Eval_URL"] = link1["href"]

            else:
                sibling1 = sibling1.find_next_sibling("div")
                link1 = sibling1.find("a")
                if link1 and link1.has_attr("href"):
                    df.loc[i, "Eval_URL"] = link1["href"]

In [28]:
# Grade_URL function
def grade_URL(soup):
    indicator2 = soup.find("div", class_="bar", string=lambda s: s and "Grade history" in s)
    if indicator2:
        sibling2 = indicator2.find_next_sibling("div")
        
        if sibling2:
            exam_term = sibling2.get_text(strip=True)[-3]
            course_term = df.loc[i,"Schedule"][0:3]
            
            if (course_term in ["Aut", "Jan"] and exam_term == "v") or (course_term in ["Spr", "Aug", "Jun"] and exam_term == "s"):
                link2 = sibling2.find("a")
                if link2 and link2.has_attr("href"):
                    df.loc[i, "Grade_URL"] = link2["href"]

            else:
                sibling2 = sibling2.find_next_sibling("div")
                link2 = sibling2.find("a")
                if link2 and link2.has_attr("href"):
                    df.loc[i, "Grade_URL"] = link2["href"]

In [29]:
# Obtains the grade_URL and eval_URL
for i in tqdm(range(len(df))):
    url = df.loc[i, "URL"] + "/info"
    soup = extract_html(url)
    grade_URL(soup)
    eval_URL(soup)

  0%|          | 0/119 [00:00<?, ?it/s]

In [30]:
# Checks if all courses have been passed grade_url and eval_url
print("Courses missing grade_URL and eval_URL:")
df[(df["Grade_URL"].isna() | (df["Grade_URL"] == "")) | (df["Eval_URL"].isna() | (df["Eval_URL"] == ""))]

Courses missing grade_URL and eval_URL:


,Term,Course code,Passed DI,Course name,Registered students,Passed students,Exam pass percentage,Average grade,ECTS,Schedule,Level,URL,Grade_URL,Eval_URL,Eval1,Eval2,Eval3,Eval4,Eval5,Workload


In [31]:
# Obtains the evaluation value(s)
for i in tqdm(range(len(df))):
    url = df.loc[i, "Eval_URL"]  # Get evaluation URL for the current course
    soup = extract_html(url)  # Parse HTML content from the evaluation page
    
#________WORKLOAD__________________________________________________
    # Extrating the workload score
    workload_scores = {
        "Meget mindre": 5,
        "Noget mindre": 7,
        "Det samme": 9,
        "Noget mere": 11,
        "Meget mere": 13
    }
    
    # The workload question text to search for
    workload_question = "5 ECTS point er normeret til 9"
    
    # Find the workload question div by searching for a unique part of the question text
    question_div = soup.find("div", class_="FinalEvaluation_QuestionText", string=lambda s: s and workload_question in s)

    if question_div:
        # The wrapper div containing all responses
        wrapper = question_div.find_parent("div", class_="CourseSchemaResultHeader")

        # All the siblings divs with class RowWrapper contain individual responses
        row_wrappers = wrapper.find_next_siblings("div", class_="RowWrapper")

        # Extract counts for each workload category
        response_counts = {}

        for row in row_wrappers:
            option_div = row.find("div", class_="FinalEvaluation_Result_OptionColumn")
            if option_div:
                option_text = option_div.text.strip()
                if option_text in workload_scores:
                    count_span = row.find("div", class_="Answer_Result_Background").find("span")
                    count = int(count_span.text.strip())
                    response_counts[option_text] = count

        # Calculate weighted average for workload
        total_responses = sum(response_counts.values())
        if total_responses > 0:
            weighted_sum = sum(workload_scores[cat] * count for cat, count in response_counts.items())
            average_workload_score = weighted_sum / total_responses
        else:
            average_workload_score = None

        # Store the average workload score in the dataframe
        df.loc[i, "Workload"] = average_workload_score

    else:
        # Question not found
        df.loc[i, "Workload"] = None
        
#_______EVALUATIONS________________________________________________________________________________________________
    
    # Mapping of Danish evaluation responses to numerical scores (0–100 scale)
    category_scores = {
        "Helt uenig": 0,
        "Uenig": 25,
        "Hverken eller": 50,
        "Enig": 75,
        "Helt enig": 100
    }
    
    # List of evaluation questions to extract scores for
    target_questions = [
        "jeg har lært meget i dette kursus.",
        "undervisningsaktiviteterne på kurset stemmer godt overens med kursets læringsmål.",
        "undervisningsaktiviteterne motiverer mig til at arbejde med stoffet.",
        "jeg i løbet af kurset har haft mulighed for at få feedback på, hvordan jeg fagligt klarer mig på kurset.",
        "det generelt har været klart for mig, hvad der forventes af mig i øvelser, projektarbejde og lignende."
    ]
    
    # For each question, extract response counts and calculate an average evaluation score
    for idx, question in enumerate(target_questions, start=1):
        response_counts = {}  # Store counts of each response category for current question
    
        # Find the div containing the exact question text (case-insensitive match)
        question_div = soup.find("div", class_="FinalEvaluation_QuestionText", string=lambda s: s and question.lower() in s.lower())
    
        if question_div:
            # Get the container div that holds all results for this question
            wrapper = question_div.find_parent("div", class_="CourseSchemaResultHeader")
            # Find all sibling divs that represent individual response options
            row_wrappers = wrapper.find_next_siblings("div", class_="RowWrapper")
    
            # Extract count of each response category (e.g., "Enig") from the HTML
            for row in row_wrappers:
                option_div = row.find("div", class_="FinalEvaluation_Result_OptionColumn")
                if option_div:
                    option_text = option_div.text.strip()
                    if option_text in category_scores:
                        count_span = row.find("div", class_="Answer_Result_Background").find("span")
                        count = int(count_span.text.strip())
                        response_counts[option_text] = count
                        # Stop if all 5 categories have been found
                        if len(response_counts) == 5:
                            break
    
        # Calculate weighted average score for the question based on response counts
        total_responses = sum(response_counts.values())
        if total_responses > 0:
            weighted_sum = sum(category_scores[cat] * count for cat, count in response_counts.items())
            average_score = weighted_sum / total_responses
        else:
            average_score = None  # No data available for this question
    
        # Save the average score in a new column (Eval1, Eval2, etc.) for the current row
        df.loc[i, f"Eval{idx}"] = average_score

  0%|          | 0/119 [00:00<?, ?it/s]

In [32]:
# Obtains the values from the grade page

# Adds exam related values (average grade, amount of passed students, percentage of passed students)
for i in tqdm(range(len(df))):
    url = df.loc[i,"Grade_URL"]
    soup = extract_html(url) #Extracts the grade page
    
    #Extracts the grade
    avg_grade = None
    indicator = soup.find('td', string=lambda text: text and "Eksamensgennemsnit" in text) # Finds the indicating label
    if indicator:
        value_cell = indicator.find_next_sibling('td') # finds the position
        avg_grade = value_cell.get_text(strip=True) # defines the string
    df.loc[i,"Average grade"] = avg_grade

    #Extracts the passed percentage of students
    passed_students = None
    indicator = soup.find('td', string=lambda text: text and "Antal b" in text) # Finds the indicating label
    if indicator:
        value_cell = indicator.find_next_sibling('td')  # Finds the position
        passed_students = value_cell.get_text(strip=True) if value_cell else None  # Defines the string if value_cell exists
    df.loc[i, "Passed students"] = passed_students
    df.loc[i, "Exam pass percentage"] = passed_students


    #Extracts the amount of students registered for the 1st exam
    registered_students = None
    indicator = soup.find('td', string=lambda text: text and "Antal tilmeldte" in text) # Finds the indicating label
    if indicator:
        value_cell = indicator.find_next_sibling('td')  # Finds the position
        registered_students = value_cell.get_text(strip=True) if value_cell else None  # Defines the string if value_cell exists
    df.loc[i, "Registered students"] = registered_students

  0%|          | 0/119 [00:00<?, ?it/s]

In [33]:
# Correction of false grade URLs

for i in tqdm(range(len(df))):
    if df.loc[i,"Average grade"] is None:
        print(f"Missing grade values detected for row index {i}")
        url = df.loc[i, "URL"] + "/info"
        soup = extract_html(url)

        # URL
        indicator2 = soup.find("div", class_="bar", string=lambda s: s and "Grade history" in s)
        sibling2 = indicator2.find_next_sibling("div")
        link2 = sibling2.find("a")["href"]

        if link2 == df.loc[i,"Grade_URL"]:
            sibling2 = sibling2.find_next_sibling("div")
            link2 = sibling2.find("a")["href"]
            df.loc[i,"Grade_URL"] = link2
        
        else:
            df.loc[i,"Grade_URL"] = link2

  0%|          | 0/119 [00:00<?, ?it/s]

Missing grade values detected for row index 12
Missing grade values detected for row index 65


In [34]:
# Correcting missing grade values
for i in tqdm(range(len(df))):
    if pd.isna(df.loc[i,"Average grade"]):
        url = df.loc[i,"Grade_URL"]
        soup = extract_html(url) #Extracts the grade page
        
        #Extracts the grade
        avg_grade = None
        indicator = soup.find('td', string=lambda text: text and "Eksamensgennemsnit" in text) # Finds the indicating label
        if indicator:
            value_cell = indicator.find_next_sibling('td') # finds the position
            avg_grade = value_cell.get_text(strip=True) # defines the string
        df.loc[i,"Average grade"] = avg_grade
    
        #Extracts the passed percentage of students
        passed_students = None
        indicator = soup.find('td', string=lambda text: text and "Antal b" in text) # Finds the indicating label
        if indicator:
            value_cell = indicator.find_next_sibling('td')  # Finds the position
            passed_students = value_cell.get_text(strip=True) if value_cell else None  # Defines the string if value_cell exists
        df.loc[i, "Passed students"] = passed_students
        df.loc[i, "Exam pass percentage"] = passed_students

        #Extracts the amount of students registered for the 1st exam
        registered_students = None
        indicator = soup.find('td', string=lambda text: text and "Antal tilmeldte" in text) # Finds the indicating label
        if indicator:
            value_cell = indicator.find_next_sibling('td')  # Finds the position
            registered_students = value_cell.get_text(strip=True) if value_cell else None  # Defines the string if value_cell exists
        df.loc[i, "Registered students"] = registered_students

  0%|          | 0/119 [00:00<?, ?it/s]

### Data cleaning

In [36]:
# Correct the Course name format
df["Course name"] = df.loc[:,"Course name"].astype(str).str[11:-2]

In [37]:
# Corrects the average grade format
for i in tqdm(range(len(df))):
    
    if df.loc[i,"Average grade"].strip()[0].isdigit():
        df.loc[i,"Average grade"] = df.loc[i,"Average grade"].split(' ')[0].replace(",", ".")
    else:
        df.loc[i,"Average grade"] = "Pass/Non-Pass"

  0%|          | 0/119 [00:00<?, ?it/s]

In [38]:
# Corrects Passed students format
for i in tqdm(range(len(df))):
    if df.loc[i, "Passed students"] == None:
        df.loc[i, "Passed students"] = np.nan
    else:
        lines = df.loc[i, "Passed students"].splitlines()
        first_number = next((line.strip() for line in lines if line.strip().isdigit()), None)
        df.loc[i, "Passed students"] = first_number

  0%|          | 0/119 [00:00<?, ?it/s]

In [39]:
# Corrects Exam pass percentage format
for i in tqdm(range(len(df))):
    if df.loc[i, "Exam pass percentage"] is None:
        df.loc[i, "Exam pass percentage"] = np.nan
    else:
        text = str(df.loc[i, "Exam pass percentage"])
        match = re.search(r"(\d+)\s*%\s*af de fremmødte", text)
        
        if match:
            df.loc[i, "Exam pass percentage"] = int(match.group(1))
        else:
            df.loc[i, "Exam pass percentage"] = np.nan

  0%|          | 0/119 [00:00<?, ?it/s]

In [40]:
df[["Evaluation score", "DI percentage"]] = ""

In [41]:
df["Evaluation score"] = df[["Eval1", "Eval2", "Eval3", "Eval4", "Eval5"]].mean(axis=1)
df["Evaluation score"] = round(df["Evaluation score"].astype(float), 1)

In [42]:
# Corrects the workload format
df["Workload"] = round(df["Workload"].astype(float), 1)

In [43]:
#Calculates the DI percentage
df["DI percentage"] = 100 * df["Passed DI"].astype(float) / df["Passed students"].astype(float)
df["DI percentage"] = round(df["DI percentage"], 1)

In [44]:
# Drops unnecesary columns
df = df.drop(["Eval1", "Eval2", "Eval3", "Eval4", "Eval5", "Eval_URL", "Grade_URL", "Term", "Passed DI", "Passed students"], axis=1)

In [45]:
# Checks for NaN values
df[df.isna().any(axis=1)]

,Course code,Course name,Registered students,Exam pass percentage,Average grade,ECTS,Schedule,Level,URL,Workload,Evaluation score,DI percentage


In [46]:
# Saves the updated data
df.to_excel("Data/data_updated.xlsx", index=False)

In [47]:
df

,Course code,Course name,Registered students,Exam pass percentage,Average grade,ECTS,Schedule,Level,URL,Workload,Evaluation score,DI percentage
0,41749,Precision Machining,32,93,7.8,5,Autumn 13-week,MSc,https://kurser.dtu.dk/course/41749,10.4,76.5,10.0
1,41525,Finite Element Methods,59,100,8.7,10,Autumn 13-week,MSc,https://kurser.dtu.dk/course/41525,10.7,84.3,5.1
2,38106,Developing an Entrepreneurial mindset,402,95,Pass/Non-Pass,5,Autumn 13-week,MSc,https://kurser.dtu.dk/course/38106,8.7,67.1,1.8
3,38106,Developing an Entrepreneurial mindset,494,96,Pass/Non-Pass,5,Spring 13-week,MSc,https://kurser.dtu.dk/course/38106,9.2,65.9,1.5
4,38109,X-Tech+,25,100,Pass/Non-Pass,10,Autumn 13-week,MSc,https://kurser.dtu.dk/course/38109,11.0,80.5,12.0
5,38109,X-Tech+,11,100,Pass/Non-Pass,10,Spring 13-week,MSc,https://kurser.dtu.dk/course/38109,11.9,95.0,27.3
6,41740,Digital Manufacturing - Industry 4.0,61,100,11,5,Autumn 13-week,MSc,https://kurser.dtu.dk/course/41740,10.8,60.1,5.0
7,41743,"Micro product design, development and production",19,100,9.4,5,January 3-week,MSc,https://kurser.dtu.dk/course/41743,10.5,94.2,15.8
8,41822,Experimental fluid mechanics,48,100,Pass/Non-Pass,5,January 3-week,MSc,https://kurser.dtu.dk/course/41822,9.6,83.7,6.4
9,42009,Introductory Economics,292,98,7.7,5,Autumn 13-week,BSc,https://kurser.dtu.dk/course/42009,8.7,66.6,1.1
